In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric
import logging

from src.utils import logging_utils

# logging_utils.configure(level=logging.DEBUG)

In [ ]:
# stats = {}

# dataset = data.load_dataset()
# for relation in dataset:
#     stats[relation.name] = {
#         "name": relation.name,
#         "category": relation.properties.relation_type,
#         "num_samples": len(relation.samples)
#     }

# with open("stats/data.json", "w") as f:
#     json.dump(stats, f, indent=2)

In [ ]:
# ############################################
# sweep_root = "../../results/sweep-24-trials"
# # sweep_root = "../../results/sweep-bare"
# model_name = "llama-13b" # gptj | gpt2-xl | llama-13b
# ############################################

# sweep_path = f"{sweep_root}/{model_name}"

In [ ]:
# from src.utils.sweep_utils import read_sweep_results


# sweep_results = read_sweep_results(sweep_path, economy=False)
# list(sweep_results.keys())

In [ ]:
# with open("stats/data.json", "r") as f:
#     stats = json.load(f)

# for relation in stats:
#     if relation not in sweep_results:
#         stats[relation][model_name] = f"--"
#         print(f"Skipping {relation} -- not found for {model_name}")
#         continue
#     trail_tests = np.array([
#         trial["n_test_samples"] 
#         for trial in sweep_results[relation]["trials"]
#     ])
#     # if len(trail_tests) < (3 if model_name != "llama-13b" else 2):
#     #     stats[relation][model_name] = f"--"
#     #     print(f"Skipping {relation} -- not enough trials for {model_name}")
#     #     continue
#     # else:
#     stats[relation][model_name] = f"{trail_tests.mean() :.2f} \pm {trail_tests.std() :.2f}"

# with open("stats/data.json", "w") as f:
#     json.dump(stats, f, indent=2)

## Table 4

In [ ]:
with open("stats/data.json", "r") as f:
    stats = json.load(f)

In [ ]:
def change_decimal(result):
    if result == "--":
        return result
    mean, std = result.split(" \pm ")
    mean = float(mean)
    std = float(std)
    return f"{mean:.1f} \pm {std:.1f}"

In [ ]:
category = "bias"
category_stats = [
    stats[relation] 
    for relation in stats 
    if stats[relation]["category"] == category
]

gpt_j_correct_count = 0
num_samples = 0

category_stats = sorted(
    category_stats, key=lambda x: x["num_samples"], reverse=True
)
print(len(category_stats))
for relation in category_stats:
    relation_stats = relation
    relation_name = relation_stats["name"]
    gptj_count = change_decimal(relation_stats["gptj"])
    gpt2_count = change_decimal(relation_stats["gpt2-xl"])
    llama_count = change_decimal(relation_stats["llama-13b"])
    print(f"& {relation_name} & ${relation_stats['num_samples']}$ & ${gptj_count}$ & ${gpt2_count}$ & ${llama_count}$ \\\\ ")

    num_samples += relation_stats["num_samples"]
    if gptj_count != "--":
        gpt_j_correct_count += float(gptj_count.split(" \pm ")[0])

print("\\hline")

len(category_stats), num_samples, f"{gpt_j_correct_count:.1f}"